In [1]:
import pandas as pd
pd.options.display.max_columns = 100
import numpy as np

import scipy.optimize as opt

from pathlib import Path
import pathlib
from tqdm.notebook import tqdm
INPUT = Path("../input/google-smartphone-decimeter-challenge/")

In [2]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

# Apply WLS on one collection and one measurement

In [3]:
# directory setting
INPUT = '../input/google-smartphone-decimeter-challenge'
p = pathlib.Path(INPUT)

base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')

# train derived
train_files = list(p.glob('train/*/*/*_derived.csv'))
print('train_derived.csv count :', len(train_files))

tds = []
for train_file in tqdm(train_files):
    tds.append(pd.read_csv(train_file))
train_derived = pd.concat(tds)
train_derived['phone'] = train_derived['collectionName'] + '_' + train_derived['phoneName']


# test derived
test_files = list(p.glob('test/*/*/*_derived.csv'))
print('test_derived.csv count :', len(test_files))

tds = []
for test_file in tqdm(test_files):
    tds.append(pd.read_csv(test_file))
test_derived = pd.concat(tds)
test_derived['phone'] = test_derived['collectionName'] + '_' + test_derived['phoneName']


# ground_truth
gt_files = list(p.glob('train/*/*/ground_truth.csv'))
print('ground_truth.csv count : ', len(gt_files))

gts = []
for gt_file in tqdm(gt_files):
    gts.append(pd.read_csv(gt_file))
ground_truth = pd.concat(gts)
ground_truth['phone'] = ground_truth['collectionName'] + '_' + ground_truth['phoneName']

train_derived.csv count : 73


  0%|          | 0/73 [00:00<?, ?it/s]

test_derived.csv count : 48


  0%|          | 0/48 [00:00<?, ?it/s]

ground_truth.csv count :  73


  0%|          | 0/73 [00:00<?, ?it/s]

# preprocessing

In [4]:
collection_name = [i for i in base_test['phone'].unique()]
phone = [i for i in base_test['phoneName'].unique()]

test_derived = test_derived[test_derived['millisSinceGpsEpoch'].isin(base_test['millisSinceGpsEpoch'])]

# Corrected pseudorange according to data instructions
test_derived['correctedPrM'] = test_derived.apply(
    lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,
    axis=1
).copy()

# Time it took for signal to travel
light_speed = 299_792_458
test_derived['transmissionTimeSeconds'] = (test_derived['correctedPrM'] / light_speed).copy()


# Compute true sat positions at arrival time
omega_e = 7.2921151467e-5
test_derived['xSatPosMRotated'] = \
    np.cos(omega_e * test_derived
['transmissionTimeSeconds']) * test_derived
['xSatPosM'] \
    + np.sin(omega_e * test_derived
['transmissionTimeSeconds']) * test_derived
['ySatPosM']
    
test_derived['ySatPosMRotated'] = \
    - np.sin(omega_e * test_derived
['transmissionTimeSeconds']) * test_derived
['xSatPosM'] \
    + np.cos(omega_e * test_derived
['transmissionTimeSeconds']) * test_derived
['ySatPosM']
    
test_derived['zSatPosMRotated'] = test_derived['zSatPosM']


# Uncertainty weight for the WLS method
test_derived['uncertaintyWeight'] = 1 / test_derived['rawPrUncM']

In [ ]:
%%time
time_uniq = test_derived['millisSinceGpsEpoch'].unique()
# fixed_pos = np.zeros(shape=(len(time_uniq), 2)) # 修正済みlat,lng格納用
dict = {}

for i, t in enumerate(tqdm(time_uniq)):
    df = test_derived[test_derived['millisSinceGpsEpoch']==t]

    # Set up least squares methods
    def distance(sat_pos, x):
        '''
        input: millisSinceGpsEpochごとにまとめられたデータフレーム
        output: 入力されたデータフレームの行数と同じ大きさのベクトル
        '''
        sat_pos_diff = sat_pos.copy(deep=True)
        sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
        sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
        sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

        sat_pos_diff['d'] = sat_pos_diff.apply(
            lambda r: r.uncertaintyWeight * 
                (np.sqrt((r.xSatPosMRotated**2 + r.ySatPosMRotated**2 + r.zSatPosMRotated**2)) + x[3] - r.correctedPrM),
            axis=1
        )
        return sat_pos_diff['d'].values

    def distance_fixed_satpos(x):
        return distance(df[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)


    # Start point for the optimiser
    x0= [0,0,0,0]

    opt_res = opt.least_squares(distance_fixed_satpos, x0)

    # Optimiser yields a position in the ECEF coordinates
    opt_res_pos = opt_res.x

    # ECEF position to lat/long
    wls_estimated_pos = ecef2lla(*opt_res_pos[:3]) # x,y,z
    wls_estimated_pos = np.squeeze(wls_estimated_pos) # １次元を削除

    # 結果を格納
    dict[t] = wls_estimated_pos[0], wls_estimated_pos[1]
    # fixed_pos[i,0] = wls_estimated_pos[0]
    # fixed_pos[i,1] = wls_estimated_pos[1]
    

In [271]:
dict

{1274119794431: (37.42441489289282, -122.09182418331804),
 1274119795431: (37.42439044540883, -122.09183743620962),
 1274119796431: (37.4244098358593, -122.09183482497366),
 1274119797431: (37.42440671944522, -122.09180600557806),
 1274119798431: (37.42442420834625, -122.09178139644061)}

In [27]:
val_baseline = base_train[
    (base_train['collectionName']==collection_name)
    & (base_train['phoneName']==phone)
    & (base_train['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

ValueError: ('Lengths must match to compare', (131342,), (73,))

In [ ]:
# datafame to check modified distance
val_base_train = val_base_train[val_base_train['millisSinceGpsEpoch'].isin(ground_truth['millisSinceGpsEpoch'])]

val_ground_truth = val_ground_truth[val_ground_truth['millisSinceGpsEpoch'].isin(base_train['millisSinceGpsEpoch'])]

In [12]:
val_groundtruth = df_groundtruth[
    (df_groundtruth['collectionName']==collection_name)
    & (df_groundtruth['phoneName']==phone)
    & (df_groundtruth['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

In [13]:
print("Baseline distance with groundtruth position (m)")
calc_haversine(val_baseline['latDeg'], val_baseline['lngDeg'], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Baseline distance with groundtruth position (m)


15.419474998287948

In [14]:
print("Our estimated position (with WLS) distance with groundtruth position (m)")
calc_haversine(wls_estimated_pos[0], wls_estimated_pos[1], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Our estimated position (with WLS) distance with groundtruth position (m)


11.161633363336618